# 爬虫基础: Requests + BeautifulSoup

“爬虫”，是访问互联网网页 ->定位网页元素 -> 爬取网页内容的过程。<br>
在实际工作中，访问的工作交给urllib或者requests完成；爬取的工作则交给xpath, BeatifulSoup乃至正则合作完成。<br>
想学习爬虫知识，首先需要了解HTTP基础请求方法：get和post

## HTTP方法：GET和POST

### GET方法

<font color='red'>请注意，查询字符串（名称/值对）是在 GET 请求的 URL 中发送的</font><br>
如：```https://www.baidu.com/s?ie=utf-8&wd=python```

有关 GET 请求的其他一些注释：

* GET 请求可被缓存
* GET 请求保留在浏览器历史记录中
* GET 请求可被收藏为书签
* GET 请求不应在处理敏感数据时使用
* GET 请求有长度限制
* GET 请求只应当用于取回数据

### POST方法

<font color='red'>请注意，查询字符串（名称/值对）是在 POST 请求的 HTTP 消息主体中发送的</font><br>
```
POST http://dcms-ml-dev.dc68032.easn.morningstar.com/automation/api/namesimilar HTTP/1.1
Host: dcms-ml-dev.dc68032.easn.morningstar.com
Connection: keep-alive
Content-Length: 103
accept: application/json
Origin: http://dcms-ml-dev.dc68032.easn.morningstar.com
User-Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36
Content-Type: application/json
Referer: http://dcms-ml-dev.dc68032.easn.morningstar.com/apidocs/
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7
{
  "country": "",
  "onlyactive": "0",
  "querytype": "share",
  "text": "S&P 500",
  "universe": ""
}
```

有关 POST 请求的其他一些注释：

* POST 请求不会被缓存
* POST 请求不会保留在浏览器历史记录中
* POST 不能被收藏为书签
* POST 请求对数据长度没有要求

![compare](./image/2018-09-17 12_15_17-HTTP 方法GET对比POST.png)

## Requests

requests是一个很实用的Python HTTP客户端库，编写爬虫和测试服务器响应数据时经常会用到。可以说，Requests 完全满足如今网络的需求。<br>
Requests的官方文档 http://docs.python-requests.org/en/master/ <br>
安装方式一般采用```pip install requests``` <br>
此文涉及dcms-ml-dev.dc68032.easn.morningstar.com站点，该站点是DCMS的restful api站点

### GET请求

In [10]:
import requests
r = requests.get('http://dcms-ml-dev.dc68032.easn.morningstar.com/automation/api/smoke')
print(r)

<Response [200]>


#### 带参数的Get请求
这里需要加入headers，给一个user-agent伪装一下自己

In [23]:
parameters = {'ie': 'utf-8', 'wd':'python'}
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"}
requestforbing = requests.get('https://www.baidu.com/s', params=parameters, headers=headers)
# print('Response: {0}, Whole URL: {1}, Text: {2}'.format(requestforbing, requestforbing.url, requestforbing.content))
print('Response: {0}, Whole URL: {1}'.format(requestforbing, requestforbing.url))

Response: <Response [200]>, Whole URL: https://www.baidu.com/s?ie=utf-8&wd=python


#### 将请求的网页内容保存到本地

In [22]:
rstream = requests.get('https://www.baidu.com/s', params=parameters, headers=headers, stream=True)
try:
    with open('./html/baiduresult.html', 'wb') as fd:
        for chunk in rstream.iter_content(1000):
            fd.write(chunk)
except Exception as e:
    print(e)

[outputhtml](./html/baiduresult.html)

### POST请求